In [1]:
from os.path import join
from glob import glob
from utils.custom_utils import RetrieveData,new_dataset
from tqdm import tqdm
import numpy as np
import pandas as pd

from astropy.io import fits
from photutils.segmentation import detect_sources, detect_threshold
import statmorph

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss,roc_auc_score, precision_recall_fscore_support

In [2]:
URL='https://tinyurl.com/hdd4kwva'

RetrieveData(URL)
new_dataset()

Reconstructing RMS:   3%|█▉                                                         | 310/9470 [00:01<00:45, 203.50it/s]/home/clement/master_thesis/utils/custom_utils.py:96: RuntimeWarning: invalid value encountered in sqrt
  total_rms=np.sqrt(rms**2+source_rms)
Reconstructing RMS: 100%|██████████████████████████████████████████████████████████| 9470/9470 [00:45<00:00, 207.62it/s]


In [3]:
root_dir='./data/new_data/'

data_path=glob(join(root_dir,'lensed/*.fits'))+glob(join(root_dir,'non_lensed/*.fits'))
category=[1 for _ in range(len(glob(join(root_dir,'lensed/*.fits'))))]+[0 for _ in range(len(glob(join(root_dir,'non_lensed/*.fits'))))]

data_mapping=np.transpose([data_path,category])
rng = np.random.default_rng()
rng.shuffle(data_mapping)

In [4]:
def corner_value(image,size=5):
    dimensions=np.shape(image)
    
    data=image[:size,:size] #up left corner
    data=np.append(data,image[:size,dimensions[0]-size:]) #up right corner
    data=np.append(data,image[dimensions[0]-size:,:size]) #down left corner
    data=np.append(data,image[dimensions[0]-size:,dimensions[1]-size:]) #down right corner
    
    return data

def galaxy_map(segm):
    label = np.argmax(segm.areas)+1
    segm_map=1*(segm.data==label)
    return segm_map

def mask(segm):
    galaxy=(galaxy_map(segm))
    segm_map=segm.data!=0
    mask=segm_map-galaxy
    bool_mask=mask==1
    return bool_mask

In [5]:
def compute_parameters(file_path,n_sigma=1.5,npixels=5):
    directory=join(*file_path.split('/')[:-1])
    filename=file_path.split('/')[-1]
    rms_path=join(directory,'RMS',filename)
    psf_path=join(directory,'PSF',filename)
    
    image, header = fits.getdata(file_path, header=True)
    rms,rms_header=fits.getdata(rms_path,header=True)
    psf,psf_header=fits.getdata(psf_path,header=True)
    
    mean_background=np.mean(corner_value(image))
    std_background=np.std(corner_value(image))
    
    threshold=detect_threshold(image,n_sigma,background=mean_background,error=std_background)
    segm = detect_sources(image, threshold, npixels=npixels)
    
    Mask=mask(segm)
    segm_map=galaxy_map(segm)
    
    weight_map=1/rms
    
    try:
        source_morphs = statmorph.source_morphology(image, segm_map, weightmap=weight_map, mask=Mask,psf=psf)
        morph = source_morphs[0]
        return morph
    except:
        return None

In [6]:
data={'file':[],'category':[],'flag_sersic':[],'flag':[],'working':[],
     'sersic_amplitude':[],'sersic_rhalf':[],'sersic_n':[],'sersic_xc':[],'sersic_yc':[],'sersic_ellip':[],'sersic_theta':[],
     'asymmetry':[],'concentration':[],'deviation':[],'gini':[],'intensity':[],'m20':[],'rpetro_circ':[],'smoothness':[]}
for file,category in tqdm(data_mapping[:2000]):
    params=compute_parameters(file)
    if params:
        data['file'].append(file)
        data['category'].append(category)
        data['flag_sersic'].append(int(params.flag_sersic))
        data['flag'].append(int(params.flag))
        data['working'].append(1)
        
        data['sersic_amplitude'].append(params.sersic_amplitude)
        data['sersic_rhalf'].append(params.sersic_rhalf)
        data['sersic_n'].append(params.sersic_n)
        data['sersic_xc'].append(params.sersic_xc)
        data['sersic_yc'].append(params.sersic_yc)
        data['sersic_ellip'].append(params.sersic_ellip)
        data['sersic_theta'].append(params.sersic_theta)
        
        data['asymmetry'].append(params.asymmetry)
        data['concentration'].append(params.concentration)
        data['deviation'].append(params.deviation)
        data['gini'].append(params.gini)
        data['intensity'].append(params.intensity)
        data['m20'].append(params.m20)
        data['rpetro_circ'].append(params.rpetro_circ)
        data['smoothness'].append(params.smoothness)
        
    else:
        data['file'].append(file)
        data['category'].append(category)
        data['flag_sersic'].append(None)
        data['flag'].append(None)
        data['working'].append(0)
        
        data['sersic_amplitude'].append(None)
        data['sersic_rhalf'].append(None)
        data['sersic_n'].append(None)
        data['sersic_xc'].append(None)
        data['sersic_yc'].append(None)
        data['sersic_ellip'].append(None)
        data['sersic_theta'].append(None)
        
        data['asymmetry'].append(None)
        data['concentration'].append(None)
        data['deviation'].append(None)
        data['gini'].append(None)
        data['intensity'].append(None)
        data['m20'].append(None)
        data['rpetro_circ'].append(None)
        data['smoothness'].append(None)
    

  8%|██████▍                                                                         | 160/2000 [01:08<11:55,  2.57it/s]/home/clement/anaconda3/envs/memoire/lib/python3.11/site-packages/astropy/modeling/functional_models.py:3109: RuntimeWarning: overflow encountered in power
  return amplitude * np.exp(-bn * (z ** (1 / n) - 1))
 35%|███████████████████████████▋                                                    | 693/2000 [04:53<10:30,  2.07it/s]/home/clement/anaconda3/envs/memoire/lib/python3.11/site-packages/astropy/modeling/functional_models.py:3109: RuntimeWarning: overflow encountered in power
  return amplitude * np.exp(-bn * (z ** (1 / n) - 1))
/home/clement/anaconda3/envs/memoire/lib/python3.11/site-packages/astropy/modeling/functional_models.py:3109: RuntimeWarning: invalid value encountered in multiply
  return amplitude * np.exp(-bn * (z ** (1 / n) - 1))
 74%|██████████████████████████████████████████████████████████                     | 1470/2000 [13:08<08:01,  1.10it/s]/h

In [7]:
df=pd.DataFrame(data)

In [8]:
df.to_csv(join(root_dir,'data.csv'),index=False)

In [9]:
#some statistics
#ratio of category
raw_ratio=len(df.loc[df['category']=='0'])/len(df)*100
print(f'dans le dataset il y a {raw_ratio} % qui sont non_lentillé et {100-raw_ratio} % de lentillé')

df_working=df.loc[df['working']==1]
working_ratio=len(df_working)/len(df)*100
print(f'le ratio de cas qui marche est de {working_ratio} % et de cas qui ne marche pas est {100-working_ratio} %.')

cat_working_ratio=len(df_working.loc[df_working['category']=='0'])/len(df_working)*100
print(f'au sein des cas qui marchent, il y a {cat_working_ratio} % de non_lentillés et {100-cat_working_ratio} % de lentillés')

print('\nOn ne prend plus que les cas qui marchent\n\n')
#flag ratio
perfect_working=len(df_working.loc[(df_working['flag_sersic']==0)].loc[df_working['flag']==0])/len(df_working)*100
print(f'il y a {perfect_working} % de cas qui marche parfaitement')
global_sersic=len(df_working.loc[df_working['flag_sersic']==1])/len(df_working)*100
print(f'il y a {global_sersic} % de flag sersic\n')

pur_sersic=len(df_working.loc[df_working['flag_sersic']==1].loc[df_working['flag']==0])/len(df_working)*100
print(f'nombre de cas ou il y a seulement un flag sersic : {pur_sersic} %')
flag1_sersic=len(df_working.loc[df_working['flag_sersic']==1].loc[df_working['flag']==1])/len(df_working)*100
flag2_sersic=len(df_working.loc[df_working['flag_sersic']==1].loc[df_working['flag']==2])/len(df_working)*100
flag4_sersic=len(df_working.loc[df_working['flag_sersic']==1].loc[df_working['flag']==4])/len(df_working)*100
print(f'sersic + flag=1 :{flag1_sersic} %')
print(f'sersic + flag=2 :{flag2_sersic} %')
print(f'sersic + flag=4 :{flag4_sersic} %\n')

pur_flag=len(df_working.loc[df_working['flag_sersic']==0].loc[df_working['flag']!=0])/len(df_working)*100
print(f'nombre de cas ou il y a seulement un flag : {pur_flag} %')
flag1=len(df_working.loc[df_working['flag_sersic']==0].loc[df_working['flag']==1])/len(df_working)*100
flag2=len(df_working.loc[df_working['flag_sersic']==0].loc[df_working['flag']==2])/len(df_working)*100
flag4=len(df_working.loc[df_working['flag_sersic']==0].loc[df_working['flag']==4])/len(df_working)*100
print(f'nombre de cas flag=1 : {flag1} %')
print(f'nombre de cas flag=2 : {flag2} %')
print(f'nombre de cas flag=4 : {flag4} %')

dans le dataset il y a 50.6 % qui sont non_lentillé et 49.4 % de lentillé
le ratio de cas qui marche est de 91.7 % et de cas qui ne marche pas est 8.299999999999997 %.
au sein des cas qui marchent, il y a 53.76226826608506 % de non_lentillés et 46.23773173391494 % de lentillés

On ne prend plus que les cas qui marchent


il y a 51.581243184296625 % de cas qui marche parfaitement
il y a 39.74918211559433 % de flag sersic

nombre de cas ou il y a seulement un flag sersic : 31.079607415485277 %
sersic + flag=1 :0.6543075245365322 %
sersic + flag=2 :8.015267175572518 %
sersic + flag=4 :0.0 %

nombre de cas ou il y a seulement un flag : 8.66957470010905 %
nombre de cas flag=1 : 1.5812431842966195 %
nombre de cas flag=2 : 7.088331515812432 %
nombre de cas flag=4 : 0.0 %


In [10]:
df_working

,file,category,flag_sersic,flag,working,sersic_amplitude,sersic_rhalf,sersic_n,sersic_xc,sersic_yc,sersic_ellip,sersic_theta,asymmetry,concentration,deviation,gini,intensity,m20,rpetro_circ,smoothness
0,./data/new_data/non_lensed/122610.fits,0,0.0,0.0,1,44.878632,3.449825,6.751547,20.991728,22.897162,0.984019,1.484627,-0.025062,3.213013,0.053572,0.565845,0.008141,-1.927345,10.214592,0.059753
1,./data/new_data/lensed/16738.fits,1,0.0,0.0,1,41.004102,6.344094,1.158116,20.444537,20.515821,0.171672,1.253265,0.103076,2.289918,0.021189,0.402943,0.469078,-1.539813,14.695033,0.000661
2,./data/new_data/non_lensed/61192.fits,0,1.0,0.0,1,0.721072,32.404456,29.403080,20.852244,21.790776,0.530427,2.466141,0.114506,3.018808,0.134927,0.555249,0.230021,-1.450642,16.259225,-0.024405
3,./data/new_data/non_lensed/117181.fits,0,0.0,0.0,1,3.338871,19.921473,5.154777,22.991493,22.009004,0.558701,2.043427,-0.063385,3.331824,0.009853,0.495785,0.022643,-2.056366,20.760156,-0.132963
4,./data/new_data/lensed/104408.fits,1,0.0,1.0,1,13.107419,9.202041,3.785384,21.682664,20.909242,0.181911,2.637884,0.068009,2.546194,0.030910,0.471605,0.201160,-1.689431,15.972006,-0.029427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,./data/new_data/non_lensed/133875.fits,0,0.0,2.0,1,4.684682,21.143303,6.826835,22.011000,22.006232,0.653476,0.720534,-0.004985,3.674292,0.014478,0.568135,0.017908,-2.207807,22.470177,-0.042161
1995,./data/new_data/non_lensed/54108.fits,0,1.0,0.0,1,52.054469,4.008899,34.022634,22.206571,21.844411,0.521143,2.627489,0.026603,3.228908,0.032592,0.563140,0.000000,-1.982073,16.794900,0.045643
1996,./data/new_data/non_lensed/106539.fits,0,0.0,0.0,1,25.743561,3.787574,5.997044,21.967496,21.069100,0.931889,2.001337,-0.032603,3.189763,0.021187,0.575054,0.000000,-1.887444,12.756307,-0.031503
1998,./data/new_data/lensed/93625.fits,1,1.0,2.0,1,1.397989,38.549129,29.440062,21.524318,21.072170,0.135761,2.686765,0.074351,3.176039,0.023595,0.559789,0.009561,-2.038586,22.151423,-0.034817


In [11]:
#test en injectant tout
mat=df_working.loc[:,df.columns!='file']
y=np.array(mat.category)
X=np.array(mat.loc[:,mat.columns!='category'])

cut1,cut2=int(0.6*len(X)),int(0.8*len((X)))
X_train,X_valid,X_test=X[:cut1],X[cut1:cut2],X[cut2:]
y_train,y_valid,y_test=y[:cut1],y[cut1:cut2],y[cut2:]

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf=clf.fit(X_train, y_train)
    
score=clf.score(X_valid,y_valid)
print(f'score={score}')
    
pred=clf.predict(X_test)
    
conf_matrix=confusion_matrix(y_test,pred)
print(f'confusion matrix: \n{conf_matrix}')
print(f'len(X_test)={len(X_test)}\n')
acc=accuracy_score(y_test,pred)
precision,recall,fscore,support=precision_recall_fscore_support(y_test,pred,average='weighted')
print(f'Accuracy: {acc*100} %',f'Precision: {precision*100} %', f'Recall: {recall*100} %',f'F-score: {fscore*100} %',sep='\n')



score=0.9073569482288828
confusion matrix: 
[[179  20]
 [ 16 152]]
len(X_test)=367

Accuracy: 90.19073569482289 %
Precision: 90.2280411855217 %
Recall: 90.19073569482289 %
F-score: 90.198644029736 %


conf matrix = [[TN,FP],
               [FN,TP]]

Accuracy= (TP+TN)/total #Number of correct prediction
Precision= TP/(TP+FP) = TP/positiv #Number of correct positive predictions out of all predicted positive
Recall= TP/(TP+FN) #Number of correct positive prediction out of really positive cases
F-score= 2*(pressicion*recall)/(precision+recall) #Harmonique mean of the precision and the recall

precision=1 ---> classifier don't make errors
recall=1 ---> classifier find all the relevant cases
F-score = 1 the best classifier